In [12]:
import carla
import pygame
import math
import random

In [14]:
# Initialize Pygame
pygame.init()
WINDOW_WIDTH, WINDOW_HEIGHT = 800, 600
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("2D Map View")

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
GRAY = (50, 50, 50)


In [15]:
# Connect to CARLA Simulator
client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
world = client.get_world()

In [16]:
# Get the vehicle blueprint and spawn it
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
spawn_points = world.get_map().get_spawn_points()
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
if vehicle is None:
    print("Failed to spawn vehicle.")
    exit()


In [17]:
# Set vehicle to autopilot
vehicle.set_autopilot(True)

In [18]:
# Attach spectator to follow the vehicle
spectator = world.get_spectator()

In [19]:
# Scaling function to fit CARLA coordinates to the 2D map
def scale_to_map(carla_location, map_width, map_height, world_extent):
    x = int((carla_location.x + world_extent) * (map_width / (2 * world_extent)))
    y = int((world_extent - carla_location.y) * (map_height / (2 * world_extent)))
    return x, y

In [20]:
# Get world extent for scaling (adjust to match your map size)
WORLD_EXTENT = 150.0  # Adjust based on the CARLA map size

In [21]:
# Draw grid on the map
def draw_grid(surface, width, height, cell_size):
    for x in range(0, width, cell_size):
        pygame.draw.line(surface, GRAY, (x, 0), (x, height))
    for y in range(0, height, cell_size):
        pygame.draw.line(surface, GRAY, (0, y), (width, y))

In [22]:
# Main loop
clock = pygame.time.Clock()
running = True

try:
    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Update vehicle position
        vehicle_transform = vehicle.get_transform()
        vehicle_location = vehicle_transform.location

        # Update spectator view to follow the vehicle
        spectator_transform = carla.Transform(
            vehicle_transform.location + carla.Location(z=50),  # Adjust height
            carla.Rotation(pitch=-90)  # Look downwards
        )
        spectator.set_transform(spectator_transform)

        # Scale vehicle location to map coordinates
        vehicle_x, vehicle_y = scale_to_map(vehicle_location, WINDOW_WIDTH, WINDOW_HEIGHT, WORLD_EXTENT)

        # Draw map
        window.fill(BLACK)
        draw_grid(window, WINDOW_WIDTH, WINDOW_HEIGHT, 50)

        # Draw the vehicle as a green rectangle
        pygame.draw.rect(window, GREEN, (vehicle_x - 5, vehicle_y - 5, 10, 10))

        # Update the display
        pygame.display.flip()

        # Limit frame rate
        clock.tick(30)

finally:
    vehicle.destroy()
    pygame.quit()